# Cleanup

After building your model you may want to delete your campaign, solutions, and datasets. The following cells will ensure that you have successfully cleaned up all of the resources you created in this lab.

## Imports and Connections to AWS

The following lines import all the necessary libraries and then connect you to Amazon Personalize.


In [ ]:
# Imports

import json
import numpy as np
import pandas as pd
import time
import os
import boto3

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
s3_client = boto3.client('s3')
iam = boto3.client("iam")


## Defining the Things to Cleanup

Using the store command we will retrieve all the values needed to cleanup our work.

In [ ]:
%store -r

##### Delete Personalize Constructs

In [ ]:
# Delete the filter:
delete_filter_response = personalize.delete_filter(
    filterArn=filter_arn
)

print(json.dumps(delete_filter_response, indent=2))

In [ ]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(5)
    try:
        describe_filter_response = personalize.describe_filter(
            filterArn=filter_arn
            )
        status = describe_filter_response['filter']['status']
    except:
        status = "DELETED"
    print("CampaignStatus: {}".format(status))
    if status == "DELETE IN_PROGRESS" or status == "DELETE PENDING" :
        continue
    else:
        break

In [ ]:
# Delete the campaign:
delete_campaign_response = personalize.delete_campaign(
    campaignArn=campaign_arn
)

print(json.dumps(delete_campaign_response, indent=2))

In [ ]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    try:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn=campaign_arn
            )
        status = describe_campaign_response['campaign']['status']
    except:
        status = "DELETED"
    print("CampaignStatus: {}".format(status))
    if status == "DELETE IN_PROGRESS" or status == "DELETE PENDING" :
        continue
    else:
        break

In [ ]:
# Delete the solution
delete_solution_response = personalize.delete_solution(
    solutionArn=solution_arn
)

print(json.dumps(delete_solution_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    try:
        describe_solution_response = personalize.describe_solution(
            solutionArn=solution_arn
        )
        status = describe_solution_response['solution']['status']
    except:
        status = "DELETED"
    print("SolutionStatus: {}".format(status))
    if status == "DELETE IN_PROGRESS" or status == "DELETE PENDING":
        continue
    else:
        break

In [ ]:
# Delete the interaction dataset
delete_interactions_dataset_response = personalize.delete_dataset(
    datasetArn=interactions_dataset_arn
)

print(json.dumps(delete_interactions_dataset_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    try:
        describe_interactions_dataset_response = personalize.describe_dataset(
            datasetArn=interactions_dataset_arn
        )
        status = describe_interactions_dataset_response['dataset']['status']
    except:
        status = "DELETED"
    print("DataSetStatus: {}".format(status))
    if status == "DELETE IN_PROGRESS" or status == "DELETE PENDING":
        continue
    else:
        break

In [ ]:
# Delete the items dataset
delete_items_dataset_response = personalize.delete_dataset(
    datasetArn=items_dataset_arn
)

print(json.dumps(delete_items_dataset_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    try:
        describe_items_dataset_response = personalize.describe_dataset(
            datasetArn=items_dataset_arn
        )
        status = describe_items_dataset_response['dataset']['status']
    except:
        status = "DELETED"
    print("DataSetStatus: {}".format(status))
    if status == "DELETE IN_PROGRESS" or status == "DELETE PENDING":
        continue
    else:
        break

In [ ]:
# Delete the interactions schema
delete_interactions_schema_response = personalize.delete_schema(
    schemaArn=interactions_schema_arn
)

print(json.dumps(delete_interactions_schema_response, indent=2))

In [ ]:
# Delete the items schema
delete_items_schema_response = personalize.delete_schema(
    schemaArn=items_schema_arn
)

print(json.dumps(delete_items_schema_response, indent=2))

In [ ]:
# Delete the Dataset Group

delete_dataset_group_response = personalize.delete_dataset_group(
    datasetGroupArn=dataset_group_arn
)

print(json.dumps(delete_dataset_group_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    try:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn=dataset_group_arn
        )
        status = describe_dataset_group_response['datasetGroup']['status']
    except:
        status = "DELETED"
    print("DataSetStatus: {}".format(status))
    if status == "DELETE PENDING" or status == "DELETE IN_PROGRESS":
        continue
    else:
        break

In [ ]:
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)

delete_role_response = iam.delete_role(
    RoleName=role_name
)

print(json.dumps(delete_role_response, indent=2))

## Empty Your S3 Bucket

Next empty your S3 bucket, you uploaded a movie file to it in the first notebook.


In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).delete()

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(items_filename).delete()

In [ ]:
if os.path.exists(interactions_filename):
  os.remove(interactions_filename)
else:
  print("The file does not exist") 

In [ ]:
if os.path.exists(items_filename):
  os.remove(items_filename)
else:
  print("The file does not exist") 

## Last Step

After cleaning up all of the resources you can now close this window and go back to the github page you stareted on. At the bottom of the Readme file are steps to delete the CloudFormation stack you created earlier. Once that is done you are 100% done with the lab.

Deleting CloudFormation Template (Manual)

1. After executing above cells, ensure the SageMaker kernel is brought down (by navigating to the circle button on the left menu).
2. Close the SageMaker Studio domain browser window.
3. Delete the CloudFormation template.
4. Due to a known issue, the EFS file share auto-created by SageMaker Studio will cause issues when executing above step. If that happens, the CloudFormation stack delete may fail after executing for a long time.
5. To successfully delete the stack, manually remove the auto-created EFS volume related to the SageMaker domain by navigating to the console. Also, recommend deleting the auto-created VPC manually so it removes all related ENIs and security groups.

Thanks for working through this content, reach out to us if you have any questions.
